# Рекомендация тарифов

In [1]:
#импорируем библиотеки
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [2]:
#читаем файл и выводим первые строки
data = pd.read_csv('/datasets/users_behavior.csv')
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


Данные в таблице годятся для дальнйшего анализа. Названия колонок в праввильном стиле. Пропусков не видно. В целом значения данных представлены наглядно и понятно. 

In [3]:
# изучим данные с помощью info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Пропусков также не обнаружено. Данные представлены вещественными числами, кроме целевого столбца is_ultra. В принципе колонку messages можно было бы привести к целым числам, но построению модели это не помещает.

Данные поверсностно изучены можно пристыпать к построению моделей.

In [4]:
#разделим выборку на обучающую, валидационную и тестовую в соотношении 3:1:1
# сначала разделим датасет на параметры и целевой параметр
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']


In [5]:
# теперь поделим выборку на обучающую(60% данных) и остальную часть (40% данных)
train_features, rest_features, train_target, rest_target = train_test_split(features,target, test_size=0.4, random_state=100)

In [6]:
#проверим количество строк в разбивка
print(len(train_features))
len(rest_features)

1928


1286

In [7]:
# теперь поделим остальную выборку на валидационную и тестовую пополам (т.е 20% от общего датасета)
valid_features, test_features, valid_target, test_target = train_test_split(rest_features, rest_target, test_size=0.5, random_state=100)

In [8]:
print(len(valid_features))
len(test_features)

643


643

Выборка разделена для модели в соотношении 3:1:1 и готова для работы с моделью.

In [12]:
# рассмотрим три модели для решения данной задачи классификации: Дерево решений, логическую регрессию и случайный лес
#начнем подбор модели с дерево решений
best_result = 0
best_model = None
depth = 0 
for i in range(1,11):
    model = DecisionTreeClassifier(random_state=100, max_depth=i)
    model.fit(train_features, train_target)
    result = model.score(valid_features, valid_target)
    if result>best_result:
        best_result=result
        best_model=model
        depth = i
        
print('accuracy:', best_result)
print('глубина древа:', depth)

accuracy: 0.8180404354587869
глубина древа: 3


На валидационной выборки метрика accuracy составляет практически 82% для древа глубной 3. Проверим другие модели.

In [15]:
# рассморим логическую регрессию
model = LogisticRegression(random_state=100)
model.fit(train_features, train_target)
result = model.score(valid_features, valid_target)
    
print('accuracy:', result)   


accuracy: 0.702954898911353


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Acccuracy данной модели ниже чем у древа всего лишь 70%.

In [14]:
# рассморим модель случайного леса
best_result_forest = 0
best_model_forest = None
depth_forest = 0 
estimators = 0
for i in range(1,11):
    for k in range(3,60,3):
        model = RandomForestClassifier(random_state=100, max_depth=i, n_estimators=k)
        model.fit(train_features, train_target)
        result = model.score(valid_features, valid_target)
        if result>best_result:
            best_result_forest=result
            best_model_forest=model
            depth_forest = i
            estimators = k
print('accuracy:', best_result_forest)
print('глубина древа:', depth_forest)
print('количество древ:', estimators)

accuracy: 0.8195956454121306
глубина древа: 9
количество древ: 15


Accuracy случайного леса тоже вышло примерно 82%, но в целом результат выше на тысячную лучше, чем у древа. Так как в данной работе важно точность то будет выбрана модель случайного леса глубиной 9 и количеством древ 15. Однако если бы мы выбирали точность с учетом быстродействия, то лучшей моделью стало бы дерево решений, так как разница метрики минимальная.

In [16]:
#проверяем модель случайного леса на тестовой выборке
model = RandomForestClassifier(random_state=100, max_depth=9, n_estimators=15)
model.fit(train_features, train_target)
testing = model.score(test_features, test_target)
if testing>0.75:
    print('Модель прошла проверку с результатом:', testing)

Модель прошла проверку с результатом: 0.7838258164852255


Данная модель имеет accuracy больше 0.75. Результат тестирования удовлетворяет запрос проекта.

In [17]:
# дополнительная проверка
#иcпользую Dummy c гиперпараметром most_frequent 
#получу модель которая предскажет наиболее частый вариант для этой бинарной класифифкации и подсчитаю accuracy
model = DummyClassifier(strategy='most_frequent', random_state=100)
model.fit(train_features, train_target)
model.score(test_features, test_target)

0.6951788491446346

Dummy-модель обладает более низкой метрикой. Проверка пройдена.